In [ ]:
from pathlib import Path
import pandas as pd
import altair as alt
from scipy.stats.mstats import gmean
import numpy as np
import re
import string

In [ ]:
# Output folders.

plot_dir = Path('./plots')
table_dir = Path('./tables')

reload_data = False
make_plots = False
make_tables = True

if make_plots:
    plot_dir.mkdir(parents=True, exist_ok=True)
if make_tables:
    table_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Combinations of width configuration mode and partial clock gating on/off.

power_modes = {'power':False, 'power_clkgated':True}

modes = {
    'Original':{'Fuse':False, 'Penalty':False},
    'Fusing':{'Fuse':True, 'Penalty':False},
    'FusingPenalty':{'Fuse':True, 'Penalty':True},
}
#mode_order = {'Original':0, 'Fusing':1, 'FusingPenalty':2}
mode_order = {'Original':0, 'Fusing':1}

df_modes = pd.DataFrame.from_dict(modes, orient='index') 
df_modes = df_modes.rename_axis('Mode').reset_index()


# Reference config for each CPU.

cpus     = ['A76', 'HP']
ref_mode = 'Original'
ref_fu   = {'A76':2, 'HP':3}

In [ ]:
# Benchmarks to include, and their name in the plot.

benchmarks = {
    #'hmmer-small_1':'hmmer',
    #'fft_simsmall_1':'fft-splash',
    'integerNN_1':'intNN',
    'streamvbyte_1':'streamvbyte',
    'img_cartoon_tiny_image1_50per':'cartoon',
    'img_cartoon_tiny_image2_50per':'cartoon',
    'img_cartoon_tiny_image3_50per':'cartoon',
    'img_cartoon_tiny_image4_50per':'cartoon',
    'img_cartoon_tiny_image5_50per':'cartoon',
    'img_canny_tiny_image1_50per':'canny',
    'img_canny_tiny_image2_50per':'canny',
    'img_canny_tiny_image3_50per':'canny',
    'img_canny_tiny_image4_50per':'canny',
    'img_canny_tiny_image5_50per':'canny',
    'img_hist_tiny_image1_50per':'hist',
    'img_hist_tiny_image2_50per':'hist',
    'img_hist_tiny_image3_50per':'hist',
    'img_hist_tiny_image4_50per':'hist',
    'img_hist_tiny_image5_50per':'hist',
    'img_integral_tiny_image1_50per':'img_integral',
    'img_integral_tiny_image2_50per':'img_integral',
    'img_integral_tiny_image3_50per':'img_integral',
    'img_integral_tiny_image4_50per':'img_integral',
    'img_integral_tiny_image5_50per':'img_integral',
    'conv_tiny_image1_50per':'conv',
    'conv_tiny_image2_50per':'conv',
    'conv_tiny_image3_50per':'conv',
    'conv_tiny_image4_50per':'conv',
    'conv_tiny_image5_50per':'conv',
    'img_erode_tiny_image1_50per':'erode',
    'img_erode_tiny_image2_50per':'erode',
    'img_erode_tiny_image3_50per':'erode',
    'img_erode_tiny_image4_50per':'erode',
    'img_erode_tiny_image5_50per':'erode',
    'img_median_tiny_image1_50per':'median',
    'img_median_tiny_image2_50per':'median',
    'img_median_tiny_image3_50per':'median',
    'img_median_tiny_image4_50per':'median',
    'img_median_tiny_image5_50per':'median',
    'amax_cols_tiny_16bit_0':'amax',
    'amax_cols_tiny_16bit_1':'amax',
    'amax_cols_tiny_16bit_2':'amax',
    'amax_cols_tiny_16bit_3':'amax',
    'amax_cols_tiny_16bit_4':'amax',
    #'asum_cols_tiny_16bit_0':'asum',
    #'asum_cols_tiny_16bit_1':'asum',
    #'asum_cols_tiny_16bit_2':'asum',
    #'asum_cols_tiny_16bit_3':'asum',
    #'asum_cols_tiny_16bit_4':'asum',
    'gemv_tiny_16bit_0':'gemv',
    'gemv_tiny_16bit_1':'gemv',
    'gemv_tiny_16bit_2':'gemv',
    'gemv_tiny_16bit_3':'gemv',
    'gemv_tiny_16bit_4':'gemv',
    #'ger_tiny_16bit_0':'ger',
    #'ger_tiny_16bit_1':'ger',
    #'ger_tiny_16bit_2':'ger',
    #'ger_tiny_16bit_3':'ger',
    #'ger_tiny_16bit_4':'ger',
    'sqnrm2_cols_tiny_16bit_0':'sqnrm2',
    'sqnrm2_cols_tiny_16bit_1':'sqnrm2',
    'sqnrm2_cols_tiny_16bit_2':'sqnrm2',
    'sqnrm2_cols_tiny_16bit_3':'sqnrm2',
    'sqnrm2_cols_tiny_16bit_4':'sqnrm2',
    'gemm_tiny_16bit_0':'gemm',
    'gemm_tiny_16bit_1':'gemm',
    'gemm_tiny_16bit_2':'gemm',
    'gemm_tiny_16bit_3':'gemm',
    'gemm_tiny_16bit_4':'gemm',
    #'fft_tiny_16bit_0':'fft',
    #'fft_tiny_16bit_1':'fft',
    #'fft_tiny_16bit_2':'fft',
    #'fft_tiny_16bit_3':'fft',
    #'fft_tiny_16bit_4':'fft',
    'fft_tiny_12bit_0':'fft',
    'fft_tiny_12bit_1':'fft',
    'fft_tiny_12bit_2':'fft',
    'fft_tiny_12bit_3':'fft',
    'fft_tiny_12bit_4':'fft',
    #'fft_tiny_8bit_0':'fft',
    #'fft_tiny_8bit_1':'fft',
    #'fft_tiny_8bit_2':'fft',
    #'fft_tiny_8bit_3':'fft',
    #'fft_tiny_8bit_4':'fft'
}

bench_thesis = {'intNN':'A1', 'streamvbyte':'A2', 'cartoon':'A3', 'sqnrm2':'K1', 'amax':'K2', 'gemv':'K3', 'gemm':'K4', 'fft':'K5', 'conv':'K6', 'median':'K7', 'img_integral':'K8', 'hist':'K9', 'canny':'K10', 'erode':'K11'}
bench_row = {'intNN':1, 'streamvbyte':1, 'cartoon':1, 'sqnrm2':1, 'amax':1, 'gemv':2, 'gemm':2, 'fft':2, 'conv':2, 'median':2, 'img_integral':5, 'hist':5, 'canny':5, 'erode':5}

bench_paper = {'intNN':'A1', 'streamvbyte':'A2', 'cartoon':'A3', 'sqnrm2':'K1', 'amax':'K2', 'gemm':'K3', 'fft':'K4', 'conv':'K5',
               'median':'K6', 'img_integral':'K7', 'canny':'K8', 'erode':'K9'}

In [ ]:
# Reload data from stats/ or get it from the already generated .csv tables 

if reload_data:
    # List all the run output folder paths.
    folder = './stats'

    runs = list(Path(folder).glob('**/roi.txt'))
    runs = [r.parents[0] for r in runs]

    # Create table with the energy and time results for the specified runs.
    ignore_cols = ['samples', 'mean', 'stdev', 'underflows', 'overflows', 'min_value', 'max_value']

    df_usage = []
    df_fusing = []

    for r in runs:
        # Params that can only be obtained from the path.
        cpu = r.parents[1].name.split('.')[0]
        pen = int(re.findall('\d?(?=pen)', r.parents[1].name)[0]) > 0

        # Params that can be obtained from config.csv.
        try:
            param = pd.read_csv(r/'config.csv', header=0)
        except:
            continue
        fu = param.iloc[0]['simdCount']
        fuse = param.iloc[0]['simdFuseCap'] > 0
        wblock = int(param.iloc[0]['widthBlockSize'])
        bench = Path(param.iloc[0]['bootscript']).stem

        if cpu in cpus and bench in benchmarks.keys():
            # Unit usage and issuing stats
            try:
                usage = pd.read_csv(r/'simd_fu_used.csv', header=0)
                issued = pd.read_csv(r/'simd_fu_issue_partial.csv', header=0)
            except:
                continue

            # Unit usage
            usage = usage.drop(columns=ignore_cols)
            usage = usage / int(usage['total'])
            usage = usage.drop(columns=['0', 'total'])
            usage['Benchmark'] = bench
            usage['CPU'] = cpu
            usage['SIMD FU'] = fu
            usage['Fuse'] = fuse
            usage['Penalty'] = pen
            usage['Width Block'] = wblock
            usage = usage.melt(id_vars=['Benchmark', 'CPU', 'SIMD FU', 'Fuse', 'Penalty', 'Width Block'],
                               var_name='Active Units', value_name='Percentage')
            df_usage.append(usage)

            # Fused insts
            issued = issued.drop(columns=(ignore_cols + ['fu', '0', 'total']))
            issued = issued.sum(axis=0)
            cycles = issued.to_numpy()
            issued = issued.index.to_numpy().astype(int)
            # fused [%] = extra / total
            fused = np.copy(issued)
            fused[0] = 0
            fusing = cycles.dot(fused) / cycles.dot(issued)
            fusing = {'Benchmark':bench, 'CPU':cpu, 'SIMD FU':fu, 'Fuse':fuse, 'Penalty':pen, 'Width Block':wblock, 'Fusing':fusing}
            df_fusing.append(fusing)

    df_usage = pd.concat(df_usage)
    df_usage = df_usage.dropna(how='all')
    df_usage = df_usage.merge(df_modes, how='inner', on=['Fuse', 'Penalty'])
    df_usage = df_usage.drop(columns=['Fuse', 'Penalty'])
    df_usage = df_usage.set_index(['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block', 'Active Units'])
    df_usage = 100 * df_usage.groupby(by=[benchmarks, None, None, None, None, None],
                                      level=['Benchmark', 'CPU', 'SIMD FU', 'Mode',
                                             'Width Block', 'Active Units']).mean()

    df_fusing = pd.DataFrame(df_fusing)
    df_fusing = df_fusing.merge(df_modes, how='inner', on=['Fuse', 'Penalty'])
    df_fusing = df_fusing.drop(columns=['Fuse', 'Penalty'])
    df_fusing = df_fusing.set_index(['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block'])
    df_fusing = 100 * df_fusing.groupby(by=[benchmarks, None, None, None, None, None],
                                        level=['Benchmark', 'CPU', 'SIMD FU', 'Mode',
                                               'Width Block']).mean()
    # Save tables.
    if make_tables:
        df_usage.to_csv('tables/simd_usage.csv')
        df_fusing.to_csv('tables/fusing.csv')
else:
    # Get stats from the already generated .csv
    df_usage = pd.read_csv('tables/simd_usage.csv', header=0,
                       index_col=['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block', 'Active Units'])
    df_fusing = pd.read_csv('tables/fusing.csv', header=0,
                            index_col=['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block'])

In [ ]:
df_usage_av = df_usage.copy()
average = df_usage_av.groupby(by=['CPU', 'SIMD FU', 'Mode', 'Width Block', 'Active Units']).mean().copy()
average = average.reset_index()
average['Benchmark'] = 'average'
average = average.set_index(['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block', 'Active Units'])
df_usage_av = df_usage_av.append(average)
df_usage_av

In [ ]:
print('For A76, the average usage of the second unit is reduced from {original:.3} to {fusing:.3}'.format(
    original=float(average.loc(axis=0)['average', 'A76', 2, 'Original', 8, 2]),
    fusing=float(average.loc(axis=0)['average', 'A76', 2, 'Fusing', 8, 2])
))
print('For HP, the average usage of the third unit is reduced from {original3:.3} to {fusing3:.3}, and the second unit from {original2:.3} to {fusing2:.3}'.format(
    original3=float(average.loc(axis=0)['average', 'HP', 3, 'Original', 8, 3]),
    fusing3=float(average.loc(axis=0)['average', 'HP', 3, 'Fusing', 8, 3]),
    original2=float(average.loc(axis=0)['average', 'HP', 3, 'Original', 8, 2:].sum()),
    fusing2=float(average.loc(axis=0)['average', 'HP', 3, 'Fusing', 8, 2:].sum())
))

In [ ]:
#config_order = ['A76-Original-2FU', 'A76-Original-1FU', 'A76-Fusing-2FU', 'A76-Fusing-1FU',
#                'HP-Original-3FU', 'HP-Original-2FU', 'HP-Original-1FU', 'HP-Fusing-3FU', 'HP-Fusing-2FU', 'HP-Fusing-1FU']
config_order = ['A76-Original-2FU', 'A76-Fusing-2FU', 'HP-Original-3FU', 'HP-Fusing-3FU']

df_usage_plot = df_usage_av.loc(axis=0)[:, :, :, :, 8].copy().reset_index()
df_usage_plot['Config'] = df_usage_plot['CPU'] + '-' + df_usage_plot['Mode'] + '-' + df_usage_plot['SIMD FU'].astype(str) + 'FU'
df_usage_plot = df_usage_plot[df_usage_plot['Config'].isin(config_order)]
df_usage_plot

In [ ]:
# Make simd usage plot. 

config_letter = {k:v for k, v in zip(config_order, string.ascii_uppercase)}
config_letter

usage_plot = alt.Chart(df_usage_plot.replace(config_letter)).mark_bar().encode(
    x=alt.X(
        'Config:N',
        sort=[config_letter[x] for x in config_order],
        axis=alt.Axis(grid=False, labelAngle=0),
        title=""
    ),
    y=alt.Y(
        'Percentage:Q',
        scale=alt.Scale(domain=(0, 100)),
        title="Execution Cycles [%]"
    ),
    color=alt.Color(
        'Active Units:O',
        scale=alt.Scale(scheme='yellowgreenblue'),
        title='Number of active SIMD units'
    ),
    column=alt.Column(
        'Benchmark:N',
        sort=list(bench_thesis.keys()),
        title="",
        spacing=8
    ),
).properties(
    width=50,
    height=250
).configure_axis(
    labelFontSize=13,
    titleFontSize=16,
    grid=True
).configure_line(
    size=2.5
).configure_legend(
    orient='bottom',
    labelFontSize=13,
    titleFontSize=14,
    titleLimit=0
)
usage_plot

In [ ]:
if make_plots:
    # Save unit usage plot.
    usage_plot.save('plots/simd_usage.svg', webdriver='firefox')

In [ ]:
df_fusing_plot = df_fusing.loc(axis=0)[list(bench_thesis.keys()), :, :, 'Fusing', 8].reset_index()
df_fusing_plot['Configuration'] = df_fusing_plot['CPU'] + '-' + df_fusing_plot['SIMD FU'].astype(str) + 'FU'
df_fusing_plot

In [ ]:
df_fusing_plot_paper = df_fusing.loc(axis=0)[list(bench_paper.keys()), :, :, 'Fusing', 8].reset_index()
df_fusing_plot_paper['Configuration'] = df_fusing_plot_paper['CPU'] + '-' + df_fusing_plot_paper['SIMD FU'].astype(str) + 'FU'
df_fusing_plot_paper

In [ ]:
order_config = ['A76-2FU', 'A76-1FU', 'HP-3FU', 'HP-2FU', 'HP-1FU']
scheme = ['#ff8533', '#ffc9a5', '#006000', '#3abf3a', '#9ae59a']

fusing_plot = alt.Chart(df_fusing_plot).mark_bar().encode(
    x=alt.X(
        'Configuration:N',
        axis=None,
        sort=order_config,
        title=""
    ),
    y=alt.Y(
        'Fusing:Q',
        title = 'Fused Instructions [%]'
    ),
    color=alt.Color(
        'Configuration:N',
        scale=alt.Scale(range=scheme),
        sort=order_config
    ),
    column=alt.Column(
        'Benchmark:N',
        sort=list(bench_thesis.keys()),
        title="",
        spacing=8
    )
).properties(
    width=50,
    height=250
).configure_axis(
    labelFontSize=13,
    titleFontSize=16,
    grid=True
).configure_line(
    size=2.5
).configure_legend(
    orient='bottom',
    labelFontSize=13,
    titleFontSize=14
)
fusing_plot

In [ ]:
order_config = ['A76-2FU', 'A76-1FU', 'HP-3FU', 'HP-2FU', 'HP-1FU']
scheme = ['#ff8533', '#ffc9a5', '#006000', '#3abf3a', '#9ae59a']

fusing_plot_paper = alt.Chart(df_fusing_plot[df_fusing_plot.Benchmark.isin(bench_paper.keys())]).mark_bar().encode(
    x=alt.X(
        'Configuration:N',
        axis=None,
        sort=order_config,
        title=""
    ),
    y=alt.Y(
        'Fusing:Q',
        title = 'Fused Instructions [%]'
    ),
    color=alt.Color(
        'Configuration:N',
        scale=alt.Scale(range=scheme),
        sort=order_config
    ),
    column=alt.Column(
        'Benchmark:N',
        sort=list(bench_paper.keys()),
        title="",
        spacing=8
    )
).properties(
    width=35,
    height=180
).configure_axis(
    labelFontSize=15,
    titleFontSize=15,
    grid=True
).configure_header(
    labelAlign='right',
    labelAngle=-40
).configure_line(
    size=2.5
).configure_legend(
    orient='bottom',
    labelFontSize=15,
    titleFontSize=15
)
fusing_plot_paper

In [ ]:
if make_plots:
    # Save plots.
    fusing_plot.save(str(plot_dir / 'fused_insts.svg'), webdriver='firefox')
    fusing_plot_paper.save(str(plot_dir / 'fused_insts_paper.svg'), webdriver='firefox')